In [ ]:
import numpy as np
import math

from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt


In [ ]:
class Parameters:
    def __init__(self, layers: list[int], random_weights = True):
        n = len(layers)
        self.W = []
        self.b = []
        for i in range(1, n):
            self.b.append(np.zeros(layers[i]))
            if random_weights: self.W.append(np.random.randn(layers[i-1], layers[i]) * 2 / math.sqrt(layers[i]))
            else: self.W.append(np.zeros((layers[i-1], layers[i])))

    def clear(self):
        self.W = [np.zeros_like(wi) for wi in self.W]
        self.b = [np.zeros_like(bi) for bi in self.b]

    def __getitem__(self, idx):
        if isinstance(idx, str):
            if idx.lower() == 'w': return self.W
            elif idx.lower() == 'b': return self.b
            else:
                raise KeyError("There is no key named " + idx)
        if isinstance(idx, tuple):
            type, i = idx
            return self[type][i]
        else: return self.W[idx], self.b[idx]

class NeuralNetwork:
    def __init__(self, layers: list[int]):
        self.depth = len(layers)
        self.parameters = Parameters(layers)
        self.grad = Parameters(layers, random_weights=False)
        self.cache = []

    def __call__(self, x: np.ndarray, flatten = False):
        if flatten:
            if x.ndim == 3: x = x.reshape(x.shape[0], -1)
            elif x.ndim == 2: x = x.reshape(-1)
        
        if x.ndim == 1: x = x.reshape(1, -1)

        if x.ndim != 2:
            raise ValueError(f"x must be 2D (ndim=2). Current shape is {x.shape}")

        w = self.parameters['w']
        b = self.parameters['b']
        self.cache = []
        for i in range(0, self.depth-1):
            self.cache.append(x)
            x = x @ w[i] + b[i]
            #Apply ReLU
            x[x < 0] = 0

        return x

    def clear_grad(self):
        self.grad.clear()
    
    def backprop(self, grad_last_layer):
        """
        Calculate the gradient for a batch and store it in self.grad

        Parameters:
            grad_last_layer: (batch, classes) representing gradient of the last layer across the whole batch
        """
        n = self.depth
        
        w = self.parameters['w']
        dz = grad_last_layer
        dw = self.grad['w']
        db = self.grad['b']
        for i in range(n-2, -1, -1):
            dw[i] = self.cache[i].T @ dz
            db[i] = dz.sum(axis=0)
            
            dz = dz @ w[i].T
            dz = dz * (self.cache[i] > 0)


    def update_with_grad(self, learning_rate = 0.01):
        #Basic optimizer
        w = self.parameters['w']
        b = self.parameters['b']
        dw = self.grad['w']
        db = self.grad['b']
        for i in range(0, self.depth - 1):
            w[i] -= learning_rate * dw[i]
            b[i] -= learning_rate * db[i]
        
        self.clear_grad()



In [ ]:
def cross_entropy_loss(logits: np.ndarray, y: np.array, return_grad = False):
    """
    Calculate the cross-entropy loss on logits (model output)

    Parameters:
        logits (np.ndarray): 2D (batch, classes) array representing logits.
        y (np.ndarray):
            if 1D (batch,): treated as the correct class indices for each batch item
            if 2D (batch, classes): treated as the one-hot vector for each batch item
        return_grad:
            if True: additionally return the gradient of the last layer w.r.t loss function
            if False: default behaviour
    
    Returns:
        when return_grad == False: return a 1D (batch,) np.ndarray representing the loss for each batch item
        when return_grad == True: return (np.ndarray, np.ndarray) where the second entry is (batch, classes) representing gradients
    """


    if logits.ndim == 1: logits = logits.reshape(1, -1)

    B = logits.shape[0]

    if y.ndim == 2:
        if not np.all((y == 0) | (y == 1)):
            raise ValueError("y must be one-hot encoded (0/1).")

        if not np.all(y.sum(axis=1) == 1):
            raise ValueError("Each row of y must contain exactly one 1.")
        
        y = np.argmax(y, axis=1)

    logits = logits - logits.max(axis=1, keepdims=True)
    exp_z = np.exp(logits)
    probs = exp_z/exp_z.sum(axis=1, keepdims=True)

    loss = -logits[np.arange(B), y] + np.log(np.sum(exp_z, axis=1))

    if return_grad == False: return loss
    
    grad = probs
    grad[np.arange(B), y] -= 1
    grad /= B

    return loss, grad

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype("float32")/255.0
x_test = x_test.astype("float32")/255.0

In [ ]:
def show_images(images, labels):
    n = len(images)
    cols = 4
    rows = math.ceil(n / 4)
    fig, axes = plt.subplots(rows, cols, figsize=(2*cols, 2*rows))
    axes = axes.flatten()

    for i in range(len(axes)):
        if (i < n):
            axes[i].imshow(images[i], cmap='gray')
            axes[i].set_title(str(labels[i]))
        axes[i].axis("off")
    #plt.tight_layout()
    plt.show()

def show_random_images(n):
    indices = np.random.choice(60000, size=n, replace=False)

    images = x_train[indices]
    labels = y_train[indices]

    show_images(images, labels)

show_random_images(8)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

mean = x_train.mean(axis=0, keepdims=True)
std  = x_train.std(axis=0, keepdims=True) + 1e-7

X_train = (x_train - mean) / std
X_test  = (x_test - mean) / std

In [ ]:
def generate_batches(batch_size = 256, x = x_train, y = y_train, shuffle = True):
    n = len(x)
    if shuffle:
        indices = np.random.permutation(n)
        shuffled_x = x[indices]
        shuffled_y = y[indices]

    for l in range(0, n, batch_size):
        r = min(l + batch_size, n)
        yield shuffled_x[l:r], shuffled_y[l:r]


In [ ]:
def accuracy(y_pred, y):
    y_pred = y_pred.argmax(axis=1)
    return np.sum(y_pred == y)

In [ ]:
model = NeuralNetwork(layers=[784, 512, 256, 128, 10])

epochs = 15

for i in range(epochs):
    print(f"Epoch {i+1}/{epochs}:", end="")

    samples_processed = 0
    total_test_accuracy = 0
    total_train_loss = 0
    for x, y in generate_batches():
        #Training:
        y_pred = model(x)
        loss, grad = cross_entropy_loss(y_pred, y, True)
        
        total_train_loss += loss.sum()

        model.backprop(grad)
        model.update_with_grad()

        samples_processed += len(x)
        print(f"\rEpoch {i}/{epochs}: {samples_processed}/{len(x_train)}", end="")
    
    y_test_pred = model(x_test)
    total_test_accuracy += accuracy(y_test_pred, y_test)

    print("\n-------------------------------")
    print(f"Loss: {total_train_loss / len(x_train)}")
    print(f"Accuracy: {total_test_accuracy / len(x_test)}")
    print()


        